# Memory

In [1]:
!pip install langchain_community

In [2]:
from langchain_openai import ChatOpenAI

In [3]:
model = ChatOpenAI(model="gpt-4o")

In [4]:
from langchain_core.messages import HumanMessage

In [5]:
model.invoke([HumanMessage(content="안녕 내 이름은 Liam이야")]).content

'안녕하세요, Liam! 만나서 반가워요. 오늘 어떻게 도와드릴까요?'

In [6]:
model.invoke([HumanMessage(content="내 이름이 뭐야?")]).content

'죄송하지만, 나는 당신의 이름을 알 수 없습니다. 당신의 이름을 알려주시면, 그 이름으로 대화할 수 있습니다. 어떻게 도와드릴까요?'

In [7]:
from langchain_core.messages import AIMessage

In [8]:
model.invoke(
    [
        HumanMessage(content="안녕 내 이름은 Liam이야"),
        AIMessage(content="반가워 Liam! 만나서 반가워요. 무엇을 도와드릴까요?"),
        HumanMessage(content="내 이름이 뭐야?"),
    ]
).content

'당신의 이름은 Liam이라고 말씀하셨죠. 다른 궁금한 점이나 도움이 필요한 것이 있나요?'

## Message History

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [10]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [11]:
config = {"configurable": {"session_id": "session_1"}}

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="안녕 내 이름은 Liam이야")],
    config=config,
)

response.content

Parent run 2455a142-ea59-4196-8d70-2226cfd02514 not found for run 436bb066-3df3-4db0-9481-85b71f1d4701. Treating as a root run.


'안녕하세요, Liam! 만나서 반가워요. 오늘 어떻게 지내고 있나요?'

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐야?")],
    config=config,
)

response.content

Parent run 22302368-e73f-417b-b640-44e8ba9aa3b5 not found for run 6d9060bb-b522-49b1-9b84-5b941bdebf1a. Treating as a root run.


'당신의 이름은 Liam이죠. 어떻게 도와드릴까요?'

In [14]:
config = {"configurable": {"session_id": "session_2"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐야?")],
    config=config,
)

response.content

Parent run 83d6d269-2ece-4fc7-96ed-d50f64b28ec3 not found for run 9242af3d-9796-475e-a902-46067a9e6506. Treating as a root run.


'죄송하지만, 당신의 이름을 알 수 있는 방법이 없습니다. 질문이 있거나 도움이 필요하면 언제든지 말씀해 주세요!'

In [15]:
config = {"configurable": {"session_id": "session_1"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐야?")],
    config=config,
)

response.content

Parent run 35b4dfe0-1eaf-4746-9023-3f330654c096 not found for run 51e0fe26-12ca-4ba2-9afe-b40b331ba681. Treating as a root run.


'당신은 Liam이라고 소개하셨어요. 맞나요?'

## Prompt templates 과 함께 쓰기

### Chain 만들기

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 도움이 되는 AI Assistant이다. 모든 질문에 최선을 다해 답변하라.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [18]:
response = chain.invoke({"messages": [HumanMessage(content="안녕 난 Liam이야")]})

response.content

'안녕하세요, Liam! 만나서 반가워요. 오늘 어떻게 도와드릴까요?'

In [19]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [20]:
config = {"configurable": {"session_id": "session_3"}}

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="안녕 난 David야")],
    config=config,
)

response.content

Parent run 78db2437-425a-4183-bf9b-184f404be2cc not found for run dbc4c89b-1149-4244-a210-4fa347b63b44. Treating as a root run.


'안녕하세요, David! 만나서 반가워요. 어떻게 도와드릴까요?'

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐야?")],
    config=config,
)

response.content

Parent run 669462e9-bfdd-4700-9135-d217270a9376 not found for run e1255758-e6a7-478e-98c4-8826081ebff0. Treating as a root run.


'당신의 이름은 David라고 하셨죠. 맞나요?'

In [72]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 도움이 되는 AI Assistant이다. 다음의 언어로 대답하라: {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [73]:
response = chain.invoke(
    {"messages": [HumanMessage(content="안녕 나는 밥이야")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

In [74]:
response = chain.invoke(
    {"messages": [HumanMessage(content="안녕 나는 밥이야")], "language": "English"}
)

response.content

'Hello, Bob! How can I assist you today?'

### RunnableWithMessageHistory 로 감싸기

In [75]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [76]:
config = {"configurable": {"session_id": "session_4"}}

In [77]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="안녕 나는 Liam이야")], "language": "Japanese"},
    config=config,
)

response.content

Parent run b6aa7e14-ca48-495f-b9e1-80068214b52a not found for run 3a010b41-dddb-4283-bb42-666309fdbe7b. Treating as a root run.


'こんにちは、Liamさん。どうぞよろしくお願いします。今日はどのようなことをお手伝いしましょうか？'

In [78]:
model.invoke(f'"{response.content}"가 한국어로 뭐야?').content

'"안녕하세요, Liam님. 잘 부탁드립니다. 오늘 어떤 것을 도와드릴까요?"'

## Conversation History 관리하기

### Chain 만들기

In [167]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    msgs = messages[-k:]
    # print(f"len msgs : {len(msgs)}")
    # [print(f"{'ai' if type(msg) == AIMessage else 'human'}: {msg.content}")for msg in msgs]
    return msgs


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [168]:
messages = [
    HumanMessage(content="안녕하세요! 저는 밥이에요"),
    AIMessage(content="안녕하세요!"),
    HumanMessage(content="저는 바닐라 아이스크림을 좋아해요"),
    AIMessage(content="좋네요"),
    HumanMessage(content="2 + 2가 뭐죠?"),
    AIMessage(content="4에요"),
    HumanMessage(content="고마워요"),
    AIMessage(content="문제 없어요!"),
    HumanMessage(content="재미있어요?"),
    AIMessage(content="네, 재미있어요!"),
]

In [169]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="내이름이 뭐야?")],
        "language": "English",
    }
)
response.content  # 기억하지 못함

len msgs : 10
ai: 안녕하세요!
human: 저는 바닐라 아이스크림을 좋아해요
ai: 좋네요
human: 2 + 2가 뭐죠?
ai: 4에요
human: 고마워요
ai: 문제 없어요!
human: 재미있어요?
ai: 네, 재미있어요!
human: 내이름이 뭐야?


'제가 당신의 이름을 알 수 있는 방법이 없어요. 당신의 이름을 알려주시겠어요?'

In [170]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="내가 좋아하는 아이스크림은 뭐야?")],
        "language": "English",
    }
)
response.content  # 기억함

len msgs : 10
ai: 안녕하세요!
human: 저는 바닐라 아이스크림을 좋아해요
ai: 좋네요
human: 2 + 2가 뭐죠?
ai: 4에요
human: 고마워요
ai: 문제 없어요!
human: 재미있어요?
ai: 네, 재미있어요!
human: 내가 좋아하는 아이스크림은 뭐야?


'당신이 좋아하는 아이스크림은 바닐라 아이스크림이에요.'

### RunnableWithMessageHistory 로 감싸기

In [171]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "session_5"}}

In [172]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="내 이름이 뭐야?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run 9b80a6cc-13d9-419d-a604-cc3d278f0188 not found for run 624e8d5b-8656-4aae-80bd-f01140c3521d. Treating as a root run.


len msgs : 10
ai: 안녕하세요!
human: 저는 바닐라 아이스크림을 좋아해요
ai: 좋네요
human: 2 + 2가 뭐죠?
ai: 4에요
human: 고마워요
ai: 문제 없어요!
human: 재미있어요?
ai: 네, 재미있어요!
human: 내 이름이 뭐야?


'죄송하지만, 저는 당신의 이름을 모릅니다. 대신 알려주시면 기억해 드릴 수 있어요.'

In [173]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="대화를 참고해서 내가 좋아하는 아이스크림은 뭐야?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run 385ef844-596c-4a1f-8309-e153ac4c1505 not found for run 521765c8-ea19-496f-8bc5-6a3b05b4b03a. Treating as a root run.


len msgs : 10
ai: 좋네요
human: 2 + 2가 뭐죠?
ai: 4에요
human: 고마워요
ai: 문제 없어요!
human: 재미있어요?
ai: 네, 재미있어요!
human: 내 이름이 뭐야?
ai: 죄송하지만, 저는 당신의 이름을 모릅니다. 대신 알려주시면 기억해 드릴 수 있어요.
human: 대화를 참고해서 내가 좋아하는 아이스크림은 뭐야?


'이전 대화에서는 당신이 좋아하는 아이스크림에 대해 언급하지 않았어요. 어떤 아이스크림을 좋아하세요?'

## Streaming

In [175]:
store.pop("session_6")

InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕하세요! 저는 밥이에요'), AIMessage(content='안녕하세요!'), HumanMessage(content='저는 바닐라 아이스크림을 좋아해요'), AIMessage(content='좋네요'), HumanMessage(content='2 + 2가 뭐죠?'), AIMessage(content='4에요'), HumanMessage(content='고마워요'), AIMessage(content='문제 없어요!'), HumanMessage(content='재미있어요?'), AIMessage(content='네, 재미있어요!'), HumanMessage(content='내 이름이 뭐야?'), AIMessage(content='죄송하지만, 저는 당신의 이름을 알 수 없습니다. 알려주시면 기억할 수 있어요!', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 122, 'total_tokens': 144}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-bdf5676b-0f56-416a-a6f5-0e821ff060c1-0', usage_metadata={'input_tokens': 122, 'output_tokens': 22, 'total_tokens': 144}), HumanMessage(content='대화를 참고해서 내가 좋아하는 아이스크림은 뭐야?'), AIMessage(content='이전 대화에서 당신이 좋아하는 아이스크림에 대해 언급한 적이 없어서 알 수 없습니다. 어떤 아이스크림을 좋아하시나요?', response_metadata={'token_usage': {'completion_tokens': 

In [176]:
config = {"configurable": {"session_id": "session_6"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="안녕 재밌는 이야기하나 해줄래?")],
        "language": "Korean",
    },
    config=config,
):
    print(r.content, end="")

Parent run 88c0758b-6660-4b5a-acfc-fef5d6bb9b85 not found for run 48e29265-3e55-490e-875d-48d6e758f2e8. Treating as a root run.


len msgs : 1
human: 안녕 재밌는 이야기하나 해줄래?
물론이죠! 다음은 재미있는 이야기입니다.

옛날 옛적에 한 작은 마을에 호기심 많고 영리한 소년이 살고 있었습니다. 그의 이름은 민준이었는데, 민준은 항상 새로운 모험을 찾아다니곤 했습니다. 어느 날, 민준은 마을 근처 숲 속 깊은 곳에 오래된 우물이 있다는 소문을 듣게 되었습니다. 그 우물에는 소원을 들어주는 마법의 물이 있다고 했습니다.

민준은 이 소문에 흥미를 느끼고 우물을 찾아 나섰습니다. 숲 속을 헤매다 마침내 민준은 오래된 우물을 발견했습니다. 우물가에 다가가자, 물 속에서 반짝이는 무언가를 발견했습니다. 그것은 바로 반짝이는 금빛 물방울이었습니다.

민준은 조심스럽게 물방울 하나를 손에 담고 소원을 빌었습니다. "나는 마을 사람들에게 행복을 가져다주는 사람이 되고 싶어!" 그 순간, 금빛 물방울이 밝게 빛나며 민준의 손에서 사라졌습니다.

그 후 민준은 마을로 돌아왔습니다. 이상하게도 마을 사람들은 하나같이 행복해 보였습니다. 그들은 서로를 도우며 웃음꽃이 피어났습니다. 민준은 자신의 소원이 이루어진 것을 깨닫고 기뻐했습니다. 그는 다시는 우물을 찾아가지 않았지만, 마을 사람들에게 행복을 전하는 마음을 계속 간직하며 살았습니다.

이렇게 민준의 용기와 소원 덕분에 마을은 영원히 행복한 곳이 되었다고 합니다.

재밌었나요?